In [1]:
import glob
import compute_bound
import re
import math
import pandas as pd

In [2]:
full_benchmarks = list(glob.glob("../benchmarks/*.fz"))

In [3]:
benchmark_keys = list(compute_bound.PAIRED_BOUNDS.keys())
benchmark_keys.extend(compute_bound.BOUNDS.keys())

In [4]:
benchmark_keys

['pos-err',
 'pos-ideal',
 'neg-err',
 'neg-ideal',
 'b1',
 'b2',
 'b3',
 'b4',
 'real',
 'ideal',
 'rel',
 'abs']

In [5]:
full_benchmarks

['benchmarks/kepler0.fz',
 'benchmarks/sum.fz',
 'benchmarks/test01_sum3.fz',
 'benchmarks/shoelace_formula.fz',
 'benchmarks/kepler1.fz',
 'benchmarks/rigidBody2.fz',
 'benchmarks/matrixDeterminant2.fz',
 'benchmarks/sineOrder3.fz',
 'benchmarks/delta.fz',
 'benchmarks/delta4.fz',
 'benchmarks/matrixDeterminant.fz',
 'benchmarks/rigidBody1.fz',
 'benchmarks/himmilbeau.fz',
 'benchmarks/kepler2.fz',
 'benchmarks/sqroot.fz']

In [6]:
def parse_bound(bnd: str):
    bnd = bnd.strip("[]")
    # print("bnd", bnd)
    bnd = bnd.strip()

    if "b" in bnd:
        mantissa = int(bnd.split("b")[0])
        exp = int(bnd.split("b")[1])
        # print("mantissa, exp)
        return mantissa * (2 ** exp)
    else:
        return float(bnd)
    # print(left, right)

In [7]:
def parse_out(filename):
    with open(filename, "r") as f:
        file = f.read()

    if "Error" in file:
        return None
        
    file = re.sub("\{.*?\}", "", file).strip()
    # try:
    bounds = [parse_bound(x.strip()) for x in file.split("in")[1].strip().split(",")]
    # except as e:
    #     print(file, e)
        # try:
        # except:
        #     print(filename, ":", file)
        
    lb = float(bounds[0])
    ub = float(bounds[1])
    return (lb, ub)


<>:8: SyntaxWarning: invalid escape sequence '\{'
<>:8: SyntaxWarning: invalid escape sequence '\{'
/var/folders/_8/qzz3btp16yl2y5nkqx7g0v0c0000gp/T/ipykernel_94748/1728640189.py:8: SyntaxWarning: invalid escape sequence '\{'
  file = re.sub("\{.*?\}", "", file).strip()


In [8]:
136015914505992771*(2**(-53))

15.1008

In [9]:
parse_out("benchmarks/kepler0-paired-ideal.g.out")

(15.1008, 210.6976)

In [10]:
ln = math.log
e = math.e

In [11]:
def bounds(result):
    bound = dict()
    try:
        (r_lb, r_ub) = result["paired-real"] # r range
        (pos_err_lb, pos_err_ub) = result["paired-pos-err"]
        (neg_err_lb, neg_err_ub) = result["paired-neg-err"]
        (pos_ideal_lb, pos_ideal_ub) = result["paired-pos-ideal"]
        (neg_ideal_lb, neg_ideal_ub) = result["paired-neg-ideal"]
        (ideal_lb, ideal_ub) = result["paired-ideal"] # ideal range
        # (abs_lb, abs_ub) = result["abs"]
    except TypeError as err:
        print(err)
        return
        
    try:
        (rel_lb, rel_ub) = result["rel"]
        print(f"relative bounds (gappa): ({rel_lb}, {rel_ub})")
    except TypeError as err:
        pass
    delta = result["fz"]
    
    r_avg = (r_lb+r_ub)/2
    
    try:
        rp_1 = abs(ln((e**delta) + ((e**delta) - (e**-delta)) * (neg_ideal_ub) / ideal_lb))
        rp_2 = abs(ln((e**-delta) + ((e**delta) - (e**-delta)) * (pos_ideal_ub) / ideal_lb))
        
        re_1 = (e**rp_1) - 1
        re_2 = (e**rp_2) - 1
        
        bound["rp_1"] = rp_1
        bound["rp_2"] = rp_2
        bound["re_1"] = re_1
        bound["re_2"] = re_2
    except ZeroDivisionError:
        pass

    try:
        rp_3 = abs(ln((e**delta) + ((e**delta) - (e**-delta)) * (neg_err_ub) / r_lb))
        rp_4 = abs(ln((e**-delta) + ((e**delta) - (e**-delta)) * (pos_err_ub) / r_lb))
    
        re_3 = (e**rp_3) - 1
        re_4 = (e**rp_4) - 1
        
        bound["rp_3"] = rp_3
        bound["rp_4"] = rp_4
        bound["re_3"] = re_3
        bound["re_4"] = re_4
    except ZeroDivisionError:
        pass

    bound["gappa-abs"] = results["abs"]
    bound["gappa-rel"] = results["rel"]
    bound["gappa-relative"] = results["relative"]
    bound["gappa-output-range"] = results["real"]
    return bound

In [12]:
e = math.e
ln = math.log

In [13]:
full_results = {}
for fz_name in full_benchmarks:
    name = fz_name.split(".")[0].split("/")[1]
    prefix = f"benchmarks/{name}"
    print(name)
    results = dict()
    
    for unpaired_bound in compute_bound.BOUNDS.keys():
        try:
            filename = f"{prefix}-{unpaired_bound}.g.out"
            results[unpaired_bound] = parse_out(filename)
        except FileNotFoundError:
            results[unpaired_bound] = None
        try:
            filename_paired = f"{prefix}-paired-{unpaired_bound}.g.out"
            results["paired-" + unpaired_bound] = parse_out(filename_paired)
        except FileNotFoundError:
            results["paired-" + unpaired_bound] = None
        
    for paired_bound in compute_bound.PAIRED_BOUNDS.keys():
        try:
            filename = f"{prefix}-paired-{paired_bound}.g.out"
            results["paired-" + paired_bound] = parse_out(filename)
        except FileNotFoundError:
            results["paired-" + paired_bound] = None

    # print(results)
    try:
        results["relative"] = parse_out(f"{prefix}-relative.g.out")
    except FileNotFoundError:
        results["relative"] = None

    with open(fz_name + ".out", "r") as f:
        fz_file = f.read()
        fz = float(fz_file.split("Relative error: ")[1].split("\n")[0])
        results["fz"] = fz
    # print(results)
    
    stats = bounds(results)
    full_results[name] = stats
    # print("abs error", results["abs"])
    # print(name, "\n", prefix)

kepler0
sum
test01_sum3
shoelace_formula
cannot unpack non-iterable NoneType object
kepler1
rigidBody2
matrixDeterminant2
sineOrder3
delta
delta4
matrixDeterminant
rigidBody1
himmilbeau
kepler2
sqroot
relative bounds (gappa): (-6.609458567160979e-16, 6.60945856716098e-16)


In [14]:
abs(-2)

2

In [17]:
pd.DataFrame(full_results)

,kepler0,sum,test01_sum3,shoelace_formula,kepler1,rigidBody2,matrixDeterminant2,sineOrder3,delta,delta4,matrixDeterminant,rigidBody1,himmilbeau,kepler2,sqroot
rp_1,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rp_2,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
re_1,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
re_2,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rp_3,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rp_4,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
re_3,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
re_4,0.0,0.0,0.000002,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gappa-abs,"(0.0, 1.2317258324401337e-13)","(0.0, 3.6637359812630166e-15)","(0.0, 1.9669532775878906e-06)",None,"(0.0, 5.381494361245132e-13)","(0.0, 3.6066261088762985e-11)","(0.0, 3.502975687297294e-12)","(0.0, 8.88600572369317e-16)","(0.0, 2.7605918257158916e-12)","(0.0, 1.3936016785009996e-13)","(0.0, 3.502975687297294e-12)","(0.0, 2.948752353404416e-13)","(0.0, 1.0000889005823412e-12)","(0.0, 2.8797170728012135e-12)","(0.0, 5.707240235963695e-16)"
gappa-rel,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"(-6.609458567160979e-16, 6.60945856716098e-16)"


In [15]:
pd.DataFrame(full_results).to_csv("results.csv")

In [56]:
full_results

{'kepler0': {'rp_1': 3.641531520770447e-14,
  'rp_2': 9.681144774730897e-14,
  're_1': 3.6415315207705135e-14,
  're_2': 9.681144774731365e-14,
  'rp_3': 3.641531520770447e-14,
  'rp_4': 9.681144774730897e-14,
  're_3': 3.6415315207705135e-14,
  're_4': 9.681144774731365e-14,
  'gappa-abs': (0.0, 1.2317258324401337e-13),
  'gappa-rel': None,
  'gappa-relative': None},
 'sum': {'rp_1': 2.997602166487927e-15,
  'rp_2': 4.107825191113088e-15,
  're_1': 3.1086244689504383e-15,
  're_2': 4.218847493575595e-15,
  'rp_3': 2.997602166487927e-15,
  'rp_4': 4.107825191113088e-15,
  're_3': 3.1086244689504383e-15,
  're_4': 4.218847493575595e-15,
  'gappa-abs': (0.0, 3.6637359812630166e-15),
  'gappa-rel': None,
  'gappa-relative': None},
 'test01_sum3': {'rp_1': 1.589459518314687e-06,
  'rp_2': 2.2252442148386007e-06,
  're_1': 1.5894607814370687e-06,
  're_2': 2.2252466906902413e-06,
  'rp_3': 1.5894599225475328e-06,
  'rp_4': 2.2252444170106633e-06,
  're_3': 1.5894611857802943e-06,
  're_4': 

In [57]:
pd.set_option("display.chop_threshold", 0)

In [68]:
pd.set_option("display.precision", 25)

In [72]:
pd.set_option('display.float_format', '{:.25g}'.format)

In [74]:
results

{'real': (0.8359375, 1.5625),
 'paired-real': (-0.65625, 1.0),
 'ideal': (0.8359375, 1.5625),
 'paired-ideal': (-0.65625, 1.0),
 'rel': (-6.609458567160979e-16, 6.60945856716098e-16),
 'paired-rel': None,
 'abs': (0.0, 5.707240235963695e-16),
 'paired-abs': (0.0, 9.922618282587337e-16),
 'paired-pos-err': (1.0, 1.0),
 'paired-pos-ideal': (1.0, 1.0),
 'paired-neg-err': (0.0, 1.65625),
 'paired-neg-ideal': (0.0, 1.65625),
 'paired-b1': (-0.65625, 1.0),
 'paired-b2': (-1.65625, 1.65625),
 'paired-b3': (-1.65625, 1.65625),
 'paired-b4': (-0.6562500000000004, 1.0),
 'relative': (-6.609458567160979e-16, 6.60945856716098e-16),
 'fz': 3.10862446895e-15}